In [1]:
from __future__ import print_function
from music21 import *
from collections import Counter, defaultdict
from ea.individual import Measure, Individual, Note
from ea.duration import Duration

import ea.initialisation as initialisation
import ea.modelTrainer as modelTrainer
import ea.musicPlayer as musicPlayer
import ea.simulation as simulation
import ea.fitness as fitness
import ea.mutation as mutation
import collections
import importlib
import music21
import numpy as np
import pandas as pd
import ea.individual as individual
import ea.duration as duration
import ea.constants as constants
import ea.util as util
import random
import nltk
import ea.fitness as fitness
import re
import ea.modelUpdater as modelUpdater


In [95]:
importlib.reload(initialisation)
importlib.reload(modelTrainer)
importlib.reload(musicPlayer)
importlib.reload(simulation)
importlib.reload(mutation)
importlib.reload(fitness)
importlib.reload(constants)
importlib.reload(util)
importlib.reload(individual)
importlib.reload(fitness)
importlib.reload(modelUpdater)

<module 'ea.modelUpdater' from 'B:\\School\\Master\\Thesis\\MasterThesis\\EvoMusicCompanion\\ea\\modelUpdater.py'>

## Get corpus

In [ ]:
coreCorpus = music21.corpus.corpora.CoreCorpus()
curr_corpus = music21.corpus.search('mozart', 'composer')

scores = []
for c in curr_corpus:
    score = c.parse()
    score = util.transpose_piece(score, 'C')
    scores.append(score)

In [ ]:
lakh_corpus = corpus.corpora.LocalCorpus('lakh')
curr_corpus = lakh_corpus.metadataBundle

scores = []
for c in curr_corpus:
    score = c.parse()
    score = util.transpose_piece(score, 'C')
    scores.append(score)

In [ ]:
wiki_corpus = corpus.corpora.LocalCorpus('wiki')
curr_corpus = wiki_corpus.metadataBundle

scores = []
for c in curr_corpus:
    score = c.parse()
    score = util.transpose_piece(score, 'C')
    scores.append(score)

In [2]:
folk_corpus = corpus.getComposer('essenFolksong')
scores = []
for p in folk_corpus:
    score = converter.parse(p)
    score = util.transpose_piece(score, 'C')
    scores.append(score)
print(len(scores))

31


In [ ]:
real_scores[0].metadata.title

In [3]:
real_scores = []
for s in scores:
    if s is None:
        continue
    for p in s.scores:
        if p is not None:
            real_scores.append(p)
    

In [ ]:
notes = modelTrainer.flatten(modelTrainer.get_pitches_per_score(real_scores))

In [ ]:
bigram = modelTrainer.get_probabilistic_matrix(modelTrainer.get_bigram_matrix(notes))

In [ ]:
trigram = modelTrainer.get_probabilistic_matrix(modelTrainer.get_trigram_matrix(notes))

In [ ]:
trigram.to_csv('./folk_trigram.csv')

In [ ]:
bigram.to_csv('./folk_bigram.csv')

In [ ]:
duration_matrix = modelTrainer.train_duration_matrix(real_scores)

In [ ]:
duration_matrix.to_csv('./folk_duration_matrix.csv')

In [ ]:
for n1_n2 in trigram:
    total_count = sum(trigram[n1_n2])
    for n3 in trigram[n1_n2].keys():
        if trigram[n1_n2][n3] != 0.0:
            trigram[n1_n2][n3] /= total_count


In [ ]:
s = real_scores[0]
notes = s.parts[0].getElementsByClass(stream.Measure).flat.notesAndRests
for n in notes:
    print(n.duration.type)

In [100]:
import csv
import os 

header = ["I", "NAME", "COMPOSER", "FITNESS", "C_TONE", "C_TONE_B", "CADENCE", "L_NOTE", "I_RES", "L_INT", "L_DUR",
              "CONS_R", "CONS_N", "PATTERN_D", "PATTERN_SD"]
file = f'./corpus-data.csv'
counter = 1
with open(file, mode='w', encoding="utf-8") as f:
    writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL, lineterminator = '\n')
    writer.writerow(header)
    for s in real_scores:
        piece = musicPlayer.music21_score_to_individual(s)
        if piece is None:
            continue
        row = [
            counter,
            s.metadata.title,
            s.metadata.composer,
            piece.fitness,
            piece.fitnesses["C_TONE"],
            piece.fitnesses["C_TONE_B"],
            piece.fitnesses["CADENCE"],
            piece.fitnesses["L_NOTE"],
            piece.fitnesses["I_RES"],
            piece.fitnesses["L_INT"],
            piece.fitnesses["L_DUR"],
            piece.fitnesses["CONS_R"],
            piece.fitnesses["CONS_N"],
            piece.fitnesses["PATTERN_D"],
            piece.fitnesses["PATTERN_SD"],
        ]
        writer.writerow(row)
        counter += 1
        

In [ ]:
musicPlayer.play_music_xml(real_scores[654])

In [101]:
len(real_scores)

7652

In [ ]:
for n in real_scores[654].parts[0].getElementsByClass("Note"):
    print(n)

In [ ]:
musicPlayer.music21_score_to_individual(real_scores[654])

In [ ]:
for i in real_scores[654].parts[0].getElementsByClass("Note"):
    print(i)

In [ ]:
for i in range(len(individuals)):
    print(f'Score {i}: {curr_corpus[i]}')
    fitness.print_fitness_values(individuals[i])
    print('-----------------')

In [ ]:
score = music21.corpus.parse('joplin/maple_leaf_rag.mxl')
scores = util.transpose_piece(score, 'C')
scores = [scores]

In [40]:
real_scores[196].show('musicxml')

In [ ]:
dur_matrix = modelTrainer.train_duration_matrix(scores)
pitch_matrix = modelTrainer.train_pitch_matrix(scores);

In [ ]:
sim = simulation.Simulation(0.5, 100)
population = sim.run_interactively()

In [ ]:
sim.run(20, None, None)

In [ ]:
sim.pitch_matrix

In [ ]:
musicPlayer.play(population[0:4])